In [1]:
import pennylane as qml
import numpy as np

## Constants
Below are constants defined for the code

In [2]:
NUM_QUBITS = 12
RABI_FREQUENCY = 50 * (10 ** 3)
OMEGA_M = 2.75 * (10 ** 6)
MU = 10 * (10 ** 3)
NUM_BLOCKS = 3

device = qml.device('default.qubit', wires=NUM_QUBITS)

## Circuit for Quantum Sensing

### Entanglement layer

$$
H_s \;=\; \sum_{i<j} J_{ij}\,\sigma_i^{\phi}\sigma_j^{\phi}
$$

$$
J_{ij} \;=\; \Omega_i \Omega_j \sum_{m} \eta_{i m}\,\eta_{j m}\,
\frac{\mu}{\mu^{2}-\omega_m^{2}}
$$

$\sigma^{\phi}_i = \cos(\phi)\sigma^x_i - \sin(\phi)\sigma^y_i$

For now assuming
- $\eta_{i m} = 0.1 * (-1)^i$
- $\Omega_{i,j} = 50 Khz$
- $\mu = 10 KHz\ to\ 20 KHz$
- $\omega_{m} = 2.75 Mhz$
- $\phi$ = trainable parameter


In [3]:
def J(i, j):
    n_im = 0.1 * ((-1)**i)
    n_jm = 0.1 * ((-1)**j)
    return RABI_FREQUENCY * RABI_FREQUENCY * n_im * n_jm * MU / (MU**2 - OMEGA_M**2)

def sigma_phi_tensor(phi, i, j):
    """Properly define σ_phi(i) ⊗ σ_phi(j)"""
    return (
        (qml.math.cos(phi)**2) * qml.prod(qml.PauliX(i), qml.PauliX(j))
        + (qml.math.sin(phi)**2) * qml.prod(qml.PauliY(i), qml.PauliY(j))
        + (qml.math.cos(phi) * qml.math.sin(phi)) *
          (qml.prod(qml.PauliX(i), qml.PauliY(j)) + qml.prod(qml.PauliY(i), qml.PauliX(j)))
    )

def entanglement_layer(phi):
    """Build an exponential of a Hamiltonian with sigma_phi tensor products"""
    coeffs = []
    observables = []
    for i in range(NUM_QUBITS):
        for j in range(i + 1, NUM_QUBITS):
            coeffs.append(J(i, j))
            observables.append(sigma_phi_tensor(phi, i, j))
    qml.exp(qml.Hamiltonian(coeffs, observables))



In [4]:
@qml.qnode(device)
def entanglement_circuit(phi):
    entanglement_layer(phi)
    return qml.probs(wires=NUM_QUBITS)


drawer = qml.draw(entanglement_circuit, show_all_wires=True)
print(drawer(np.pi/4))
print(sigma_phi_tensor(np.pi/4, 1, 2))

 0: ─╭Exp(1.00 𝓗)─┤       
 1: ─├Exp(1.00 𝓗)─┤       
 2: ─├Exp(1.00 𝓗)─┤       
 3: ─├Exp(1.00 𝓗)─┤       
 4: ─├Exp(1.00 𝓗)─┤       
 5: ─├Exp(1.00 𝓗)─┤       
 6: ─├Exp(1.00 𝓗)─┤       
 7: ─├Exp(1.00 𝓗)─┤       
 8: ─├Exp(1.00 𝓗)─┤       
 9: ─├Exp(1.00 𝓗)─┤       
10: ─├Exp(1.00 𝓗)─┤       
11: ─╰Exp(1.00 𝓗)─┤       
12: ──────────────┤  Probs
0.5000000000000001 * (X(1) @ X(2)) + 0.4999999999999999 * (Y(1) @ Y(2)) + 0.5 * (X(1) @ Y(2) + Y(1) @ X(2))


### Encoder and Decoder circuit
In Encoder circuit the order of operations are
1. Single qubit parameterized Rx's rotations
2. Interaction from exponentiated hamiltonian of Zig Zag

For decoder it is the same but in opposite order

In [5]:
def encoder_circuit(theta):
    single_rotation_param, entanglement_rotation_param = theta
    
    # Single qubit rotations layer
    for qubit in range(NUM_QUBITS):
        qml.RX(single_rotation_param, wires=qubit)
        
    # Entanglement layer    
    entanglement_layer(entanglement_rotation_param)

def decoder_circuit(nu):
    single_rotation_param, entanglement_rotation_param = nu 
    
    # Entanglement layer    
    entanglement_layer(entanglement_rotation_param)
    
    # Single qubit rotations layer
    for qubit in range(NUM_QUBITS):
        qml.RX(single_rotation_param, wires=qubit)


### Sensor circuit
It consists of
1. Sequence of RY gates
2. Parameterized encoder layer repeated NUM_BLOCKS times.
3. Sensing layer
4. Parameterized decoder layer repeated NUM_BLOCK times.
5. Sequence of RX gates

In [6]:
@qml.qnode(device)
def sensor_circuit(phi: np.complex128, theta, nu):
    for qubit in range(NUM_QUBITS):
        qml.RY(np.pi/2, wires = qubit)

    # Encoder blocks 
    for t in theta:
        encoder_circuit(t)

    for qubit in range(NUM_QUBITS):
        qml.exp(qml.PauliZ(qubit), coeff= -1j * phi)

    # Decoder circuit
    for n in nu:
        decoder_circuit(n)

    for qubit in range(NUM_QUBITS):
        qml.RX(np.pi/2, wires = qubit)

    return qml.probs(wires=range(NUM_QUBITS))

In [7]:
drawer = qml.draw(sensor_circuit, show_all_wires=False)
theta = [[np.pi/4, np.pi/4]  for _ in range(NUM_BLOCKS + 1)]
print(drawer(1.0, theta, theta))

 0: ──RY(1.57)──RX(0.79)─╭Exp(1.00 𝓗)──RX(0.79)─╭Exp(1.00 𝓗)──RX(0.79)─╭Exp(1.00 𝓗)──RX(0.79) ···
 1: ──RY(1.57)──RX(0.79)─├Exp(1.00 𝓗)──RX(0.79)─├Exp(1.00 𝓗)──RX(0.79)─├Exp(1.00 𝓗)──RX(0.79) ···
 2: ──RY(1.57)──RX(0.79)─├Exp(1.00 𝓗)──RX(0.79)─├Exp(1.00 𝓗)──RX(0.79)─├Exp(1.00 𝓗)──RX(0.79) ···
 3: ──RY(1.57)──RX(0.79)─├Exp(1.00 𝓗)──RX(0.79)─├Exp(1.00 𝓗)──RX(0.79)─├Exp(1.00 𝓗)──RX(0.79) ···
 4: ──RY(1.57)──RX(0.79)─├Exp(1.00 𝓗)──RX(0.79)─├Exp(1.00 𝓗)──RX(0.79)─├Exp(1.00 𝓗)──RX(0.79) ···
 5: ──RY(1.57)──RX(0.79)─├Exp(1.00 𝓗)──RX(0.79)─├Exp(1.00 𝓗)──RX(0.79)─├Exp(1.00 𝓗)──RX(0.79) ···
 6: ──RY(1.57)──RX(0.79)─├Exp(1.00 𝓗)──RX(0.79)─├Exp(1.00 𝓗)──RX(0.79)─├Exp(1.00 𝓗)──RX(0.79) ···
 7: ──RY(1.57)──RX(0.79)─├Exp(1.00 𝓗)──RX(0.79)─├Exp(1.00 𝓗)──RX(0.79)─├Exp(1.00 𝓗)──RX(0.79) ···
 8: ──RY(1.57)──RX(0.79)─├Exp(1.00 𝓗)──RX(0.79)─├Exp(1.00 𝓗)──RX(0.79)─├Exp(1.00 𝓗)──RX(0.79) ···
 9: ──RY(1.57)──RX(0.79)─├Exp(1.00 𝓗)──RX(0.79)─├Exp(1.00 𝓗)──RX(0.79)─├Exp(1.00 𝓗)──RX(0.79) ···
10: ──RY(1.57)──RX(0

# Optimizations